In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


documents=pd.read_csv('jd_source_c_2.csv')

documents['documents'] = documents['job_desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
documents_df=pd.DataFrame(documents,columns=['documents'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\icyro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
documents_df

,documents_cleaned
0,requirements at least months experience open consider applicants experience technical skills any programming language c c java net python good analytical problem skills business level proficiency english liaise end users l support
1,software engineer data engineer permanent position requirements degree information technology computer science biotechnology science equivalent at least years research experience field computer science data preparation mathematical modelling simulation data analytics optimization knowledge machine learning data mining techniques one areas statistical modeling methods time series natural language image video text mining optimization information retrieval proficient python excel scala r matlab proficient data science algorithms strong understanding field have experience web development html css js familarity linux able work independently develop operate app hadoop environment familiar spark hbase hive hadoop big data related technologies experience rhel linux rdbms sql essential experience cloud computing technologies azure aws knowledge windows server c added advantage good understanding building scalable system practical hands on experience data platforms tool strong critical thinking problem solving programming skills computer science knowledge interested applicants please send updated resume feroz shellinfotech com please mention you re present expected salary availability
2,senior software engineers deliver high quality functional applications websites satisfy business needs requirements clients you actively participate software development lifecycle debug applications configure existing systems we like people love code history building programs applications you actively research creative solutions problems responsibilities develop deploy fully functional applications based business technical specifications analyze overall development work plan tasks maximum efficiency reducing risk write clean testable code including unit tests debug applications websites oversee team s technical deliverables reviewing code coaching improve skills work multiple client partner teams implement cohesive end to end experiences document development operational procedures requirements degree computer science engineering equivalent years professional software development experience demonstrable ability build end to end solution using enterprise technologies e g net java salesforce etc experience creating enterprise level applications familiarity agile environments excellent troubleshooting problem solving skills good communication skills kindly indicate last drawn salary notice period
3,please note due security nature client interested applicants must singaporean pr about our client they specialist cyber security product company r d division formed talented group cyber security experts responsible delivering cutting edge projects using innovative active defense methods about security engineer working alongside research team responsible developing poc transforming research security products it expectation continuously develop knowledge specific field cyber security order establish domain expert duties include software development technical design reviews integration testing documentation addition acting technical advisor research department experience required proficient high level programming scripting language java net c python bash full stack developer database presentation layer complete sdlc experience research product development environment a keen knowledge interest cyber security understanding threat landscape e g mitm ransomware attack vectors active defence experienced software security reviews vulnerability assessment ability multi task engaged various projects simultaneously relevant security certifications would desirable understanding network layer concepts if interested learn job role job opportunities please apply job advertisement alternatively contact following consultant daryl th

In [5]:
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split()))

In [6]:
documents_df=pd.DataFrame(documents_df,columns=['documents_cleaned'])

In [9]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

### Corpus Dataframe

In [10]:
documents_df['documents_cleaned']

0      requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### Tf-idf vectors

In [19]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

In [17]:
tfidf_vectors.shape

(118, 64)

In [29]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.         0.         0.         0.26874799 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.36638399 0.         0.         0.
 0.         0.2962928  0.27171355 0.         0.23044369 0.
 0.         0.         0.33033348 0.         0.         0.
 0.         0.         0.23498792 0.         0.         0.
 0.21560272 0.         0.         0.         0.         0.
 0.44778186 0.         0.         0.         0.         0.29442962
 0.         0.         0.         0.28424261 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


##### Every document has been converted into a 64 dimensional vector. As we set the max_features=64

### Pairwise similarity

##### Pairwise cosine similarity would just be the dot product of the vectors becasue tf-idf vectors from sklearn are already normalized and L2 norm of these vectors is 1. So denominator of cosine similiarity formula is 1 in this case.

In [30]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences=euclidean_distances(tfidf_vectors)

In [31]:
print (tfidf_vectors[0])
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

[0.         0.         0.         0.26874799 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.36638399 0.         0.         0.
 0.         0.2962928  0.27171355 0.         0.23044369 0.
 0.         0.         0.33033348 0.         0.         0.
 0.         0.         0.23498792 0.         0.         0.
 0.21560272 0.         0.         0.         0.         0.
 0.44778186 0.         0.         0.         0.         0.29442962
 0.         0.         0.         0.28424261 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
(118, 118)
[1.         0.1507925  0.51638553 0.11075066 0.84839137 0.30674492
 0.29193753 0.11840473 0.17906158 0.11075066 0.11075066 0.34910695
 0.26710783 0.19913315 0.43710286 0.12393251 0.14838997 0.19344567
 0.23792929 0.32341131 0.09421267 0.17373079 0.12008003 0.16727807
 0.28889262 0.16505627 0.17671157 0.26710783 0.52245714 0.22670202
 0.1972059

##### similarity is highest, 1 at index 0 becasue they are the same documents

In [32]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents_cleaned"]}')
    print ('\n')
    print (f'Similar Documents using {matrix}:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents_cleaned"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [33]:
most_similar(1,pairwise_similarities,'Cosine Similarity')

Document: software engineer  data engineer   permanent position requirements  degree information technology  computer science  biotechnology  science equivalent  at least   years research experience field computer science  data preparation  mathematical modelling simulation  data analytics optimization  knowledge machine learning data mining techniques one areas statistical modeling methods  time series  natural language  image video text mining  optimization  information retrieval  proficient python  excel  scala  r  matlab  proficient data science algorithms strong understanding field  have experience web development  html  css  js   familarity linux  able work independently develop   operate app hadoop environment  familiar spark  hbase  hive  hadoop big data related technologies  experience rhel linux  rdbms sql essential  experience cloud computing technologies  azure aws   knowledge windows server             c  added advantage  good understanding building scalable system  practi

In [34]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support


Similar Documents using Cosine Similarity:


Document: prefer   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  soft skills  willing learn treasury tool face customers  good analytical  problem skills business level proficiency english liaise end users l  support interested candidates  kindly submit updated cv subject head title junior developer   we regret inform shortlisted candidates notified 
Cosine Similarity : 0.848391369282737


Document:   basic programming knowledge experience  gained commercially academically    good knowledge c c   java  net sql    candidates interested build career programming apply necessary training provided    good commun

In [35]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support


Similar Documents using Euclidean Distance:


Document: prefer   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  soft skills  willing learn treasury tool face customers  good analytical  problem skills business level proficiency english liaise end users l  support interested candidates  kindly submit updated cv subject head title junior developer   we regret inform shortlisted candidates notified 
Euclidean Distance : 0.5506516697827457


Document:   basic programming knowledge experience  gained commercially academically    good knowledge c c   java  net sql    candidates interested build career programming apply necessary training provided    good com

#### word2vec embeddings

In [36]:
from keras.preprocessing.text import Tokenizer
import gensim
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\icyro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
# tokenize and pad every document to make them of the same size
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1

print (tokenized_paded_documents[0])

[   6  246   99 1185    1  338  927  129    1   22    8  629   15  182
    7    7    9   49  103   30  267  116    8   26  111  247  248  630
   61  281  631   33    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [38]:
#document_word_embeddings.shape
tokenized_paded_documents.shape

(118, 64)

In [40]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),118))
words=tfidfvectoriser.get_feature_names()

for i in range(len(tokenized_paded_documents)):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
    

NameError: name 'embedding_matrix' is not defined

In [41]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [42]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support


Similar Documents using Cosine Similarity:


Document:   proficient wpf  c   net  oo design    design simple  efficient intuitive ui   attention detail   able debug complex event driven application   understanding computing science fundamentals   concurrency  algorithms  operating systems    mathematical aptitude ambition learn mathematics financial derivatives   process tools produce well written low defect rate code  collaborative development   at least   years depth commercial c  rich development experience    at least   years experience ui design implementation using wpf   experience developing supporting mission critical applications    knowledge fx advantage appreciation product structure workflow    experience working mature

In [147]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: 1. Pre-requisite Analyst Programmer 1.1 Required Competencies a. Experience four following skills: Java, EJB, JSP, JSF, Hibernate, JEE 5 above, Struts, PL/SQL, Oracle database version 11g 12c, XML b. Experience handling system-to-system backend interfaces highly preferred c. Experience software database design, Weblogic application server, UML modelling tools preferred d. Proficient spoken written English e. Good interpersonal communication skills. 1.2 Required Working Experience a. Minimum 3 years experience four following skills: Java, EJB, JSP, JSF, Hibernate, JEE 5 above, Struts, PL/SQL, Oracle database version 11g 12c, XML b. Knowledge Experience Procurement processes fin

#### Glove embeddings

In [43]:
# reading Glove word embeddings into a dictionary with "word" as key and values as word vectors

embeddings_index = dict()

with open('glove.6B.100d.txt',encoding='utf8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    

In [44]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 

embedding_matrix=np.zeros((vocab_size,100))

for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [45]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),100))
words=tfidfvectoriser.get_feature_names()

for i in range(documents_df.shape[0]):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

c:\Users\icyro\anaconda3\envs\cuda_3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [46]:
document_embeddings.shape

(118, 100)

In [47]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [48]:
most_similar(2,pairwise_similarities,'Cosine Similarity')

Document: senior software engineers deliver high quality functional applications websites satisfy business needs requirements clients  you actively participate software development lifecycle  debug applications  configure existing systems  we like people love code history building programs applications  you actively research creative solutions problems  responsibilities   develop deploy fully functional applications based business technical specifications   analyze overall development work plan tasks maximum efficiency reducing risk   write clean  testable code including unit tests    debug applications websites   oversee team s technical deliverables  reviewing code coaching improve skills   work multiple client partner teams implement cohesive end to end experiences   document development operational procedures requirements   degree computer science  engineering equivalent      years professional software development experience   demonstrable ability build end to end solution using e

In [49]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support


Similar Documents using Euclidean Distance:


Document: prefer   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  soft skills  willing learn treasury tool face customers  good analytical  problem skills business level proficiency english liaise end users l  support interested candidates  kindly submit updated cv subject head title junior developer   we regret inform shortlisted candidates notified 
Euclidean Distance : 0.8285753146738027


Document: job description  the objective position develop improved version current video analytics platform  vap  used upcoming strategic projects  job responsibilities    responsible development  implementation mainte

#### Doc2vec model

In [50]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [51]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(documents_df.documents_cleaned)]

In [52]:
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

In [203]:
document_embeddings=np.zeros((documents_df.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

C:\Users\icyro\AppData\Local\Temp\ipykernel_1144\1686857689.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings[i]=model_d2v.docvecs[i]


In [204]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [205]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.8679393547760148


Document: • Basic programming knowledge experience (gained commercially academically) • Good knowledge C/C++/Java/.NET/SQL. • Candidates interested build career programming apply necessary training provided. • Good commu

In [206]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 9.441442365283498


Document: • Basic programming knowledge experience (gained commercially academically) • Good knowledge C/C++/Java/.NET/SQL. • Candidates interested build career programming apply necessary training provided. • Good comm

#### BERT model

In [207]:
from sentence_transformers import SentenceTransformer

In [208]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [209]:
document_embeddings = sbert_model.encode(documents_df['documents_cleaned'])

In [227]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [228]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Cosine Similarity:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Cosine Similarity : 0.8988029360771179


Document: You possess following qualities: • Degree/Diploma Computer Science, Engineering related subject. • More 5 years working experience developing applications Ruby Rails. • Proven working experience software develo

In [229]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: Requirements At least 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Good analytical, problem skills business level proficiency English liaise end users L1 support


Similar Documents using Euclidean Distance:


Document: Prefer 6 months Experience , Open consider applicants experience Technical Skills: Any programming language (C, C++, Java, .Net, Python) Soft Skills: Willing learn Treasury tool face customers. Good analytical, problem skills business level proficiency English liaise end users L1 support Interested candidates, kindly submit updated CV subject head title Junior Developer . We regret inform shortlisted candidates notified.
Euclidean Distance : 7.035074710845947


Document: You possess following qualities: • Degree/Diploma Computer Science, Engineering related subject. • More 5 years working experience developing applications Ruby Rails. • Proven working experience software devel

In [230]:
most_similar(2,pairwise_similarities,'Cosine Similarity')

Document: Senior Software Engineers deliver high-quality functional applications websites satisfy business needs requirements clients. You actively participate software development lifecycle, debug applications, configure existing systems. We like people love code history building programs applications. You actively research creative solutions problems. Responsibilities • Develop deploy fully functional applications based business technical specifications • Analyze overall development work plan tasks maximum efficiency reducing risk • Write clean, testable code including unit tests. • Debug applications websites • Oversee team’s technical deliverables, reviewing code coaching improve skills • Work multiple client partner teams implement cohesive end-to-end experiences • Document development operational procedures Requirements • Degree Computer Science, Engineering equivalent • 5+ years professional software development experience • Demonstrable ability build end-to-end solution using e

In [213]:
documents_df['documents_cleaned'][0]

'requirements at least   months experience   open consider applicants experience technical skills  any programming language  c  c    java   net  python  good analytical  problem skills business level proficiency english liaise end users l  support'

In [214]:
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embeddings = embed(
    documents_df['documents_cleaned']
    )
print(embeddings)

tf.Tensor(
[[-0.01185223 -0.06793223 -0.04051131 ... -0.05885008 -0.03884387
  -0.05826777]
 [-0.04404059 -0.05294157 -0.03157094 ...  0.05280953 -0.05271323
  -0.04876457]
 [-0.01129406 -0.05722681  0.04076611 ... -0.04094793 -0.05719187
  -0.02274043]
 ...
 [-0.0057514  -0.05346822  0.05332123 ...  0.05349443 -0.05348276
  -0.04850665]
 [-0.05301718 -0.05269119  0.03394029 ...  0.00408999 -0.05810668
  -0.04789482]
 [ 0.04162162 -0.06161624  0.05892245 ... -0.01172911 -0.06092782
   0.01005125]], shape=(118, 512), dtype=float32)


In [215]:
from scipy.spatial import distance
for i in range (len(documents_df)):
    for j in range(len(documents_df)):
        print(i,distance.cosine(embeddings[i],embeddings[j]))
    #print(1-distance.cosine(embeddings[0],embeddings[i]))
"""print(1 - distance.cosine(embeddings[0], embeddings[1]))
print(1 - distance.cosine(embeddings[0], embeddings[2]))
print(1 - distance.cosine(embeddings[1], embeddings[2]))"""

0 0
0 0.49777233600616455
0 0.488195538520813
0 0.5252120792865753
0 0.169405996799469
0 0.6547227501869202
0 0.4789978265762329
0 0.5106122195720673
0 0.5063356757164001
0 0.5359683930873871
0 0.5360748171806335
0 0.49241167306900024
0 0.5030980110168457
0 0.5387088656425476
0 0.6287432312965393
0 0.5388046503067017
0 0.5091853141784668
0 0.5209928750991821
0 0.610043853521347
0 0.4802553057670593
0 0.43362748622894287
0 0.6031397581100464
0 0.5829626321792603
0 0.6165658235549927
0 0.5882691442966461
0 0.6112649738788605
0 0.5505801737308502
0 0.5030980706214905
0 0.3088375926017761
0 0.5299824476242065
0 0.6006824374198914
0 0.6139590740203857
0 0.38424450159072876
0 0.5110594928264618
0 0.5965851247310638
0 0.6772853136062622
0 0.7510556578636169
0 0.6055138111114502
0 0.4777751564979553
0 0.4757818579673767
0 0.5311248302459717
0 0.5582129061222076
0 0.428932249546051
0 0.4464910626411438
0 0.4673488140106201
0 0.5379319787025452
0 0.6507532000541687
0 0.5030980706214905
0 0.47334

'print(1 - distance.cosine(embeddings[0], embeddings[1]))\nprint(1 - distance.cosine(embeddings[0], embeddings[2]))\nprint(1 - distance.cosine(embeddings[1], embeddings[2]))'

In [307]:
from scipy.spatial import distance
for i in range (len(documents_df)):
    print(1-distance.cosine(embeddings[0],embeddings[i]))
"""print(1 - distance.cosine(embeddings[0], embeddings[1]))
print(1 - distance.cosine(embeddings[0], embeddings[2]))
print(1 - distance.cosine(embeddings[1], embeddings[2]))"""

1
0.5022276639938354
0.511804461479187
0.4747879207134247
0.830594003200531
0.34527724981307983
0.5210021734237671
0.48938778042793274
0.49366432428359985
0.4640316069126129
0.46392518281936646
0.5075883269309998
0.4969019889831543
0.4612911343574524
0.3712567687034607
0.46119534969329834
0.4908146858215332
0.47900712490081787
0.38995614647865295
0.5197446942329407
0.5663725137710571
0.3968602418899536
0.41703736782073975
0.3834341764450073
0.4117308557033539
0.3887350261211395
0.4494198262691498
0.4969019293785095
0.6911624073982239
0.47001755237579346
0.39931756258010864
0.38604092597961426
0.6157554984092712
0.4889405071735382
0.40341487526893616
0.3227146863937378
0.24894434213638306
0.3944861888885498
0.5222248435020447
0.5242181420326233
0.4688751697540283
0.44178709387779236
0.571067750453949
0.5535089373588562
0.5326511859893799
0.46206802129745483
0.3492467999458313
0.4969019293785095
0.5266526937484741
0.35581085085868835
0.5551143884658813
0.3723748028278351
0.57223486900329

'print(1 - distance.cosine(embeddings[0], embeddings[1]))\nprint(1 - distance.cosine(embeddings[0], embeddings[2]))\nprint(1 - distance.cosine(embeddings[1], embeddings[2]))'